# This notebook is used to train and test a model

# Some constants

In [1]:
nrows = 100
ncolumns = 100
batch_size = 50
color_mode = 'grayscale'
input_shape = (nrows, ncolumns, 1)
nclass = 24

# Define Model

In [2]:
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

def from_paper(input_shape=(100,100,1),nclass=24):
    model = Sequential()
    
    model.add(Conv2D(filters=16,
                     input_shape=input_shape,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=32,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3, 3),
                     activation='relu',
                     padding='same'))
    
    model.add(MaxPooling2D(pool_size=(5, 5)))
    
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(nclass, activation='softmax'))
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

Using TensorFlow backend.


# Specify the Model

In [3]:
model = from_paper((100,100,1), 24)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 64)         

In [4]:
picture_main_dir = '../dataset/'

print("Data source: " + picture_main_dir)

Data source: ../dataset/


# Specifying Training, Validation and Testing data

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen =  ImageDataGenerator(rescale=1.0/255, 
                                    rotation_range=15,
                                    zoom_range=0.2)

val_datagen =  ImageDataGenerator(rescale=1.0/255)

test_datagen =  ImageDataGenerator(rescale=1.0/255)

In [6]:
train_gen = train_datagen.flow_from_directory(directory= picture_main_dir + 'train/',
                                              target_size=(nrows, ncolumns),
                                              color_mode=color_mode,
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed=0)
val_gen = val_datagen.flow_from_directory(directory= picture_main_dir + 'val/',
                                          target_size=(nrows, ncolumns),
                                          color_mode=color_mode,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=False,
                                          seed=0)

test_gen = test_datagen.flow_from_directory(directory= picture_main_dir + 'test/',
                                            target_size=(nrows, ncolumns),
                                            color_mode=color_mode,
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            shuffle=False,
                                            seed=0)

Found 24000 images belonging to 24 classes.
Found 4800 images belonging to 24 classes.
Found 4358 images belonging to 24 classes.


# Training
### Run this section only if you want to train a model, alternatively go to next section and load weights immediately
#### By default this section is skipped

In [7]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
filepath = 'weight file.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', 
                        verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor='val_loss', 
                      mode='min', 
                      patience=40, restore_best_weights=True)

callbacks_list = [checkpoint, early]


In [8]:
# switch to True if you want to train.
if False:
    history = model.fit_generator(train_gen,
                              steps_per_epoch=nclass*1000/batch_size,          
                              validation_data=val_gen,
                              validation_steps=batch_size, 
                              epochs=40, verbose=1,
                              callbacks=callbacks_list)


# Load Weights to the Model

In [9]:
# Load weights
model.load_weights('from_paper_model_weight.h5')

# Test the model

In [10]:
model.evaluate_generator(val_gen, verbose=1)

96/96 [==============================] - 6s 58ms/step


[4.141217232245253e-06, 0.8777083158493042]

In [11]:
model.evaluate_generator(test_gen, verbose=1)

88/88 [==============================] - 4s 46ms/step


[4.172323428974778e-07, 0.8561266660690308]

In [12]:
model.evaluate_generator(train_gen, verbose=1)

480/480 [==============================] - 60s 124ms/step


[0.00013196583313401788, 0.9981250166893005]